# import json
import os
import openai
from openai import OpenAI



# optional; defaults to `os.environ['OPENAI_API_KEY']`
openai.api_key = "lm-studio"

# all client options can be configured just like the `OpenAI` instantiation counterpart
openai.base_url = "http://127.0.0.1:1234/v1/"

MODEL_1 = "meta-llama-3.1-8b-instruct"
MODEL_2 = "granite-3.1-8b-instruct"
def run_first_llm(tasks_dict):
    """
    Process tasks using the first LLM and output the intermediate results.

    Args:
        tasks_dict (dict): A dictionary where keys are task IDs (strings) and values are file paths to JSON tasks.

    Returns:
        dict: A dictionary mapping task IDs to intermediate results.
    """
    output = {}

    for task_id, task_file in tasks_dict.items():
        with open(task_file, 'r', encoding='utf-8') as file:
            task_data = json.load(file)

        prompt = f"""
        This json: {json.dumps(task_data)}. is a task. Give an English description describing what transformations you need to do
        to this grid to solve this specific task. I don't need code; your description will be used to write code.
        """
        response = openai.completions.create(
                model=MODEL_1,  # Use an appropriate model
                prompt=prompt,
                temperature=0.5,
                max_tokens=800
            )

        # Extract and store the generated result
        result = response.choices[0].text.strip()
        output[task_id] = {"description": result, "original_json": task_data}

    return output


# Dit model doet echt totaal niet wat ik wil, maar hij wil niet werken met 2x 3.1 8b voor een of andere reden.
def run_second_llm(intermediate_results):
    """
    Process the output of the first LLM using a second LLM.

    Args:
        intermediate_results (dict): A dictionary where keys are task IDs and values are intermediate results.

    Returns:
        dict: A dictionary mapping task IDs to final results.
    """
    final_output = {}

    for task_id, data in intermediate_results.items():
        description = data["description"]
        original_json = data["original_json"]

        prompt = f"""
        Here is a JSON task: {json.dumps(original_json)}.
        Based on this description of transformations: "{description}", directly apply the described transformations to the JSON task.
        Return the modified JSON as the output, reflecting the solution to the task.
        """
        response = openai.completions.create(
                model=MODEL_2,  # Use an appropriate model
                prompt=prompt,
                temperature=0.4,
                max_tokens=1200
            )

        # Extract and store the final result
        result = response.choices[0].text.strip()
        final_output[task_id] = {"generated_code": result}

    return final_output

def load_tasks(directory):
    """
    Load all JSON files from a directory and map them to task IDs.

    Args:
        directory (str): The path to the directory containing JSON task files.

    Returns:
        dict: A dictionary mapping task IDs to file paths.
    """
    tasks = {}
    for file_name in os.listdir(directory):
        if file_name.endswith('.json'):
            task_id = os.path.splitext(file_name)[0]
            tasks[task_id] = os.path.join(directory, file_name)
    return tasks

# Example usage
if __name__ == "__main__":
    tasks_directory = 'onetask'  
    tasks_dict = load_tasks(tasks_directory)
    
    # Step 1: Process tasks with the first LLM
    intermediate_results = run_first_llm(tasks_dict)
    
    # Step 2: Process intermediate results with the second LLM
    final_results = run_second_llm(intermediate_results)
    
    # Output the final results
    print(intermediate_results)
    print(json.dumps(final_results, indent=2))
